In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score, recall_score, confusion_matrix, matthews_corrcoef
from sklearn.feature_selection import RFE
from utils import score
import pickle

In [2]:
label = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/train_y_answer.csv')
ds = pd.read_csv(f"E:/Datasets/Fintech/AggregatedData/timeSeriesStats.csv")
train_ds = ds[ds.alert_key.isin(label.alert_key)]
pub_ds = ds[~ds.alert_key.isin(label.alert_key)]

Add label

In [3]:
sar_flags = []
for i in range(len(train_ds)):
    ak = train_ds.iloc[i]['alert_key']
    sar_flag = label[label.alert_key==ak].sar_flag.item()
    sar_flags.append(sar_flag)
train_ds['sar_flag'] = sar_flags

c:\Users\PoChen\anaconda3\envs\tiles\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
skf = StratifiedKFold(n_splits=5)
feat_freq = np.zeros(train_ds.iloc[:,:-2].values.shape[1])
X = train_ds.iloc[:,:-2].values
y = train_ds.iloc[:,-1].values
for idx, (train_idx, val_idx) in enumerate(skf.split(X,y)):
    train_ds_cv = train_ds.iloc[train_idx,:]
    val_ds_cv = train_ds.iloc[val_idx,:]
    X_train = train_ds_cv.iloc[:,:-2].values
    X_test = val_ds_cv.iloc[:,:-2].values
    y_train = train_ds_cv.iloc[:,-1].values
    y_test = val_ds_cv.iloc[:,-1].values
    
    #'''
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)
    #'''

    for i in range(5):
        estimator = ExtraTreesClassifier()
        selector = RFE(estimator=estimator, step=10, n_features_to_select=100)
        selector.fit(X_train, y_train)
        feat_freq = feat_freq + (0+selector.support_)
top_feats = np.argsort(feat_freq)[-100:]

In [4]:
# with open('./tsStats_result/top_feats', 'wb') as f:
#     pickle.dump(top_feats, f)

with open('./tsStats_result/top_feats', 'rb') as f:
    top_feats = pickle.load(f)

In [6]:
skf = StratifiedKFold(n_splits=5)

y_true = []
y_pred = []
fold_result = []

X = train_ds.iloc[:,:-2].values
y = train_ds.iloc[:,-1].values
for idx, (train_idx, val_idx) in enumerate(skf.split(X,y)):
    train_ds_cv = train_ds.iloc[train_idx,:]
    val_ds_cv = train_ds.iloc[val_idx,:]
    X_train = train_ds_cv.iloc[:,:-2].values
    X_val = val_ds_cv.iloc[:,:-2].values
    y_train = train_ds_cv.iloc[:,-1].values
    y_val = val_ds_cv.iloc[:,-1].values
    ak_train = train_ds_cv.iloc[:,-2:]
    ak_val = val_ds_cv.iloc[:,-2:]
    
    #'''
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_val = sc.transform(X_val)
    #'''

    X_train = X_train[:,top_feats]
    X_val = X_val[:,top_feats]
     
    if idx==0:
        print(X_train.shape)
        print(X_val.shape)
    clf = SVC(C=1e+0, class_weight='balanced', kernel='rbf', probability=True)
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_val)
    y_pred = ak_val.iloc[:,0:1] # alert_key
    y_pred['prob'] = y_prob[:,1]
    print(f"Fold-{idx+1}, score: {score(ak_val, y_pred)}")



(17528, 100)
(4382, 100)
Fold-1, score: (0.010851217747769471, 45, 4146)
Fold-2, score: (0.010578279266572637, 45, 4253)
Fold-3, score: (0.010707635009310988, 46, 4295)
Fold-4, score: (0.012089356110381078, 46, 3804)
Fold-5, score: (0.011417225117895259, 46, 4028)
